
## MSDS 451 - Final Project Compact Pipeline (5 ETFs)-Yahui Qian


In [6]:
# What this cell does:
# 1) Download 10-year daily data for WTI + 5 ETFs (USO, SPY, GLD, TLT, VGT)
# 2) Build 40/80 EMA momentum features, with 1-day lag to avoid look-ahead
# 3) Cross-asset "vote" probability for target (WTI), trade on USO
# 4) Backtest at thresholds 0.50/0.60/0.70 with simple costs; print metrics
# 5) Monte Carlo block bootstrap of strategy returns (Sharpe/MaxDD distribution)
# 6) Apply 2/20 fee model vs SPY benchmark; save plots & outputs


# %% Imports & Config
!pip install yfinance
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set matplotlib defaults (no specific colors per course rules)
plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["axes.grid"] = True

# ---- User switches ----
FETCH_FROM_YF = True   # If False, read local CSVs named like "<name>_daily_data.csv"
SAVE_DIR = Path("./outputs"); SAVE_DIR.mkdir(parents=True, exist_ok=True)

# Universe (5 assets + WTI)
TICKERS = {
    "wti": "CL=F",   # WTI Crude (Futures)
    "uso": "USO",    # Tradable proxy
    "spy": "SPY",    # US equities broad market
    "gld": "GLD",    # Gold
    "tlt": "TLT",    # Long-term Treasuries
    "vgt": "VGT",    # Tech (growth tilt)
}

START_DATE = "2015-07-31"
END_DATE   = "2025-07-31"

# EMA parameters & thresholds
EMA_FAST = 40
EMA_SLOW = 80
RISK_THRESHOLDS = {"Aggressive (0.50)":0.50, "Moderate (0.60)":0.60, "Conservative (0.70)":0.70}
TCOST_BPS = 5.0   # one-way 5 bps per position change
ANNUALIZATION = 252
BETA_TARGET = 1.0 # weight for target (WTI) own momentum in vote probability

# MC params
MC_ITER = 1000
MC_BLOCK = 10
MC_THR = 0.60     # use Moderate as the default

# Fees
MGMT_FEE = 0.02
PERF_FEE = 0.20
BENCH = "spy"     # Benchmark for fee/alpha comparison


# %% Helpers
def safe_find_col(cols, candidates):
    for c in candidates:
        if c in cols: return c
    raise ValueError(f"None of {candidates} found in {cols}")

def load_local_csv(name: str) -> pd.DataFrame:
    path = Path(f"./{name}_daily_data.csv")
    if not path.exists():
        raise FileNotFoundError(f"Local CSV not found: {path}")
    df = pd.read_csv(path)
    dcol = safe_find_col(df.columns, ["Date","date","DATE","timestamp"])
    pcol = safe_find_col(df.columns, ["Close","close","adj_close","Price","price"])
    out = df[[dcol,pcol]].copy()
    out.columns = ["Date","Close"]
    out["Date"] = pd.to_datetime(out["Date"])
    return out.sort_values("Date").dropna().reset_index(drop=True)

def fetch_yf(name: str, ticker: str) -> pd.DataFrame:
    import yfinance as yf
    df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
    df = df.reset_index()[["Date","Close"]]
    df.columns = ["Date","Close"]
    return df.sort_values("Date").dropna().reset_index(drop=True)

def compute_ema_momentum(df: pd.DataFrame, span_fast=EMA_FAST, span_slow=EMA_SLOW) -> pd.DataFrame:
    p = df.copy()
    p["EMA_fast"] = p["Close"].ewm(span=span_fast, adjust=False).mean()
    p["EMA_slow"] = p["Close"].ewm(span=span_slow, adjust=False).mean()
    p["mom_sig"]  = (p["EMA_fast"] > p["EMA_slow"]).astype(int)
    return p[["Date","Close","mom_sig"]]

def block_bootstrap(x: np.ndarray, block_size: int, n: int, rng: np.random.Generator) -> np.ndarray:
    m = len(x); idx = []
    while len(idx) < n:
        start = rng.integers(0, m)
        block = [(start+i) % m for i in range(block_size)]
        idx.extend(block)
    return x[np.array(idx[:n])]

def backtest(ret: pd.Series, prob: pd.Series, thr: float, tcost_bps: float) -> pd.DataFrame:
    df = pd.DataFrame({"ret": ret.values, "prob": prob.values})
    df["signal"]   = (df["prob"] >= thr).astype(int)
    df["position"] = df["signal"].shift(1).fillna(0.0)
    df["prev_pos"] = df["position"].shift(1).fillna(0.0)
    df["turnover"] = (df["position"] - df["prev_pos"]).abs()
    tcost = tcost_bps/10000.0
    df["strategy_ret"] = df["position"]*df["ret"] - tcost*df["turnover"]
    df["equity"] = (1.0 + df["strategy_ret"]).cumprod()
    return df

def perf_summary(bt: pd.DataFrame, label: str) -> dict:
    daily = bt["strategy_ret"].dropna()
    if len(daily)==0:
        return {"Label":label,"CAGR":np.nan,"Sharpe":np.nan,"MaxDD":np.nan,"Trades":0,"WinRate":np.nan}
    eq = bt["equity"].fillna(1.0).values
    peak = np.maximum.accumulate(eq)
    cagr = eq[-1]**(ANNUALIZATION/len(daily)) - 1
    sharpe = daily.mean()/(daily.std()+1e-12)*np.sqrt(ANNUALIZATION) if daily.std()>0 else np.nan
    maxdd = (eq/peak - 1.0).min()
    trades = int((bt["turnover"]>0 & (bt["position"]==1)).sum())
    winrate = (daily>0).mean()
    return {"Label":label,"CAGR":cagr,"Sharpe":sharpe,"MaxDD":maxdd,"Trades":trades,"WinRate":winrate}

def apply_fees(strategy_ret: pd.Series, bench_ret: pd.Series, mgmt_fee=MGMT_FEE, perf_fee=PERF_FEE) -> pd.DataFrame:
    df = pd.DataFrame({"strategy_ret":strategy_ret.values, "bench_ret":bench_ret.values})
    # daily management fee
    df["after_mgmt"] = df["strategy_ret"] - mgmt_fee/ANNUALIZATION
    # performance fee on positive daily excess (daily approximation)
    df["excess"] = df["after_mgmt"] - df["bench_ret"]
    df["after_fees_ret"] = df["after_mgmt"] - perf_fee*np.clip(df["excess"], 0, None)
    for col in ["strategy_ret","after_fees_ret","bench_ret"]:
        df[f"equity_{col}"] = (1.0 + df[col]).cumprod()
    return df


# %% Load data (fetch or local)
data = {}
for name, ticker in TICKERS.items():
    if FETCH_FROM_YF:
        df = fetch_yf(name, ticker)
    else:
        df = load_local_csv(name)
    data[name] = compute_ema_momentum(df)

# Align & build core series
core = None
for name, df in data.items():
    tmp = df.rename(columns={"Close":f"{name}_px","mom_sig":f"{name}_sig"})
    core = tmp if core is None else core.merge(tmp, on="Date", how="outer")
core = core.sort_values("Date").reset_index(drop=True)

# Build prob using cross-asset signals (exclude target WTI), lag by 1 day
target_sig = "wti_sig"
vote_cols = [c for c in core.columns if c.endswith("_sig") and c != target_sig]
for c in vote_cols + [target_sig]:
    core[c] = core[c].shift(1)
N = len(vote_cols)
core["votes"] = core[vote_cols].sum(axis=1)
core["prob"] = (core["votes"] + BETA_TARGET*core[target_sig])/(N + BETA_TARGET)

# Trade asset return = USO
core["ret_trade"] = core["uso_px"].pct_change()
core = core.dropna().reset_index(drop=True)

# Save core for reuse
core_path = SAVE_DIR / "core_series.csv"
core[["Date","ret_trade","prob"]].to_csv(core_path, index=False)
print(f"Saved core series: {core_path}  rows={len(core)}")


# %% Backtest at thresholds & plot equity curves
results = []
equity_curves = pd.DataFrame({"Date": core["Date"]})
for label, thr in RISK_THRESHOLDS.items():
    bt = backtest(core["ret_trade"], core["prob"], thr, tcost_bps=TCOST_BPS)
    equity_curves[label] = bt["equity"].values
    results.append(perf_summary(bt, label))

res = pd.DataFrame(results).set_index("Label").sort_index()
print("\n=== Performance Summary (5-ETF Universe) ===")
print(res.to_string(float_format=lambda x: f"{x:,.4f}"))

# Plot
plt.figure()
for label in RISK_THRESHOLDS.keys():
    plt.plot(equity_curves["Date"], equity_curves[label], label=label)
plt.title("Equity Curves (Trade: USO, Prob: Cross-Asset EMA Votes)")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.legend()
plt.tight_layout()
eq_path = SAVE_DIR / "equity_curves_5ETF.png"
plt.savefig(eq_path, dpi=150); plt.close()
print(f"Saved: {eq_path}")


# %% Monte Carlo (block bootstrap) on strategy returns for the chosen threshold
rng = np.random.default_rng(42)
thr = MC_THR
bt0 = backtest(core["ret_trade"], core["prob"], thr, tcost_bps=TCOST_BPS)
sr = bt0["strategy_ret"].dropna().values

CAGR_list, Sharpe_list, MaxDD_list = [], [], []
for _ in range(MC_ITER):
    sr_bs = block_bootstrap(sr, block_size=MC_BLOCK, n=len(sr), rng=rng)
    eq = np.cumprod(1.0 + sr_bs)
    daily = sr_bs
    cagr = eq[-1]**(ANNUALIZATION/len(daily)) - 1
    sharpe = daily.mean()/(daily.std()+1e-12)*np.sqrt(ANNUALIZATION) if daily.std()>0 else np.nan
    peak = np.maximum.accumulate(eq)
    mdd = (eq/peak - 1.0).min()
    CAGR_list.append(cagr); Sharpe_list.append(sharpe); MaxDD_list.append(mdd)

mc = pd.DataFrame({"CAGR":CAGR_list, "Sharpe":Sharpe_list, "MaxDD":MaxDD_list})
mc_path = SAVE_DIR / "mc_summary_5ETF.csv"
mc.to_csv(mc_path, index=False)
print(f"Saved MC summary: {mc_path}")

plt.figure()
plt.hist(mc["Sharpe"].dropna(), bins=40)
plt.title(f"Monte Carlo Sharpe (thr={thr})"); plt.xlabel("Sharpe"); plt.ylabel("Freq")
plt.tight_layout()
mc_sharpe_path = SAVE_DIR / "mc_sharpe_hist_5ETF.png"
plt.savefig(mc_sharpe_path, dpi=150); plt.close()

plt.figure()
plt.hist(mc["MaxDD"].dropna(), bins=40)
plt.title(f"Monte Carlo MaxDD (thr={thr})"); plt.xlabel("MaxDD"); plt.ylabel("Freq")
plt.tight_layout()
mc_dd_path = SAVE_DIR / "mc_drawdown_hist_5ETF.png"
plt.savefig(mc_dd_path, dpi=150); plt.close()
print(f"Saved: {mc_sharpe_path}, {mc_dd_path}")


# %% 2/20 Fees vs SPY benchmark
# Build daily strategy returns from bt0 (align with SPY)
bench_px = core["spy_px"].values
bench_ret = pd.Series(bench_px).pct_change().reindex_like(core["ret_trade"]).fillna(0.0).values
fees_df = apply_fees(bt0["strategy_ret"].reindex_like(core["ret_trade"]).fillna(0.0),
                     pd.Series(bench_ret).fillna(0.0),
                     mgmt_fee=MGMT_FEE, perf_fee=PERF_FEE)
fees_df["Date"] = core["Date"].values

# Metrics after fees
daily = fees_df["after_fees_ret"].dropna()
cagr_af = fees_df["equity_after_fees_ret"].iloc[-1]**(ANNUALIZATION/len(daily)) - 1 if len(daily)>0 else np.nan
shp_af = daily.mean()/(daily.std()+1e-12)*np.sqrt(ANNUALIZATION) if daily.std()>0 else np.nan
peak = np.maximum.accumulate(fees_df["equity_after_fees_ret"].values)
mdd_af = (fees_df["equity_after_fees_ret"].values/peak - 1.0).min()

with open(SAVE_DIR / "after_fees_metrics_5ETF.txt","w") as f:
    f.write(f"Threshold: {thr}\n")
    f.write(f"CAGR_after_fees: {cagr_af:.4%}\n")
    f.write(f"Sharpe_after_fees: {shp_af:.3f}\n")
    f.write(f"MaxDD_after_fees: {mdd_af:.2%}\n")

# Plot Before/After Fees vs Benchmark
plt.figure()
plt.plot(fees_df["Date"], fees_df["equity_strategy_ret"], label="Strategy (Before Fees)")
plt.plot(fees_df["Date"], fees_df["equity_after_fees_ret"], label="Strategy (After 2/20 Fees)")
plt.plot(fees_df["Date"], fees_df["equity_bench_ret"], label="Benchmark (SPY)")
plt.title(f"Before/After Fees vs Benchmark (thr={thr})")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.legend()
plt.tight_layout()
fees_plot = SAVE_DIR / "after_fees_equity_5ETF.png"
plt.savefig(fees_plot, dpi=150); plt.close()
print(f"Saved: {fees_plot}")

print("\nAll done. Outputs saved under ./outputs")


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


/var/folders/kr/f58f4vdx13g_lwx179mtw12c0000gn/T/ipykernel_5271/3864050468.py:81: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/var/folders/kr/f58f4vdx13g_lwx179mtw12c0000gn/T/ipykernel_5271/3864050468.py:81: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/var/folders/kr/f58f4vdx13g_lwx179mtw12c0000gn/T/ipykernel_5271/3864050468.py:81: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/var/folders/kr/f58f4vdx13g_lwx179mtw12c0000gn/T/ipykernel_5271/3864050468.py:81: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/var/folders/kr/f58f4vdx13g_lwx179mtw12c0000gn/T/ipykernel_5

Saved core series: outputs/core_series.csv  rows=2507

=== Performance Summary (5-ETF Universe) ===
                      CAGR  Sharpe   MaxDD  Trades  WinRate
Label                                                      
Aggressive (0.50)   0.0720  0.3830 -0.6072      15   0.4324
Conservative (0.70) 0.0325  0.2764 -0.3410      43   0.1835
Moderate (0.60)     0.0228  0.2204 -0.5754      33   0.3594
Saved: outputs/equity_curves_5ETF.png
Saved MC summary: outputs/mc_summary_5ETF.csv
Saved: outputs/mc_sharpe_hist_5ETF.png, outputs/mc_drawdown_hist_5ETF.png
Saved: outputs/after_fees_equity_5ETF.png

All done. Outputs saved under ./outputs
